# Project 3 Report: Peter Morgan, Bruce Lam, Eda Kavlakoglu

In [ ]:
# Importing Modules
import json
from pyspark.sql import Row
from pyspark.sql.functions import udf, from_json
from pyspark.sql.types import StructType, StructField, StringType

## Executive Summary

The data engineering team at Renai

## Repository Description
* docker-compose.yml
* game_api.py
* Project_3.ipynb

## Detailed Pipeline Breakdown

### Set Up
Below we will describe in detail how the pipeline is spun up. First, we need to change into the w205 directory and clone the project repository. This is done using the commands below.

```console
cd ~/w205
git clone https://github.com/mids-w205-martin-mims/project-3-superpeter55.git
```

Next, we must move into our new directory and create a new branch "assignemnt" and switch over to that branch. This is done using the commands below. We also include the git status command and its output to ensure that we are on the assignment branch as expected. As you can see, we successfully created and switched to the assignment branch

```console
cd project-3-superpeter55
git branch assignment
git checkout assignment
git status
```
```console
On branch assignment
```

The next step is going to be copying our docker-compose file from the week 13 content so we can run our docker images. This is done using the command below. In this case, the period "." at the end of the command pastes the copied file in the current directory. Once this is copied in, there are a few changes we have to make to ensure our docker containers run properly. First, we must comment out the two lines in the cloudera service that read "ports:" and "8888:8888". These lines are already commented out in the spark service in the docker-compose file and must be uncommented. This insures that we can run a jupyter notebook with pyspark in port 8888.

```console
cp ~/w205/course-content/13-Understanding-Data/docker-compose.yml .
```

Now that we have copied in the docker-compose file and made the necessary modifications, we will spin up our cluster using the command below. The -d at the end runs the containers in the background and we are able to continue to use our console. Next we will use docker-compose ps to make sure all the containers are up. The command lists all the containers and some basic information on them. Since all the containers say they are in the state "up", we will proceed.

```console
docker-compose up -d
```

Now that our cluster is running, we will create our kafka topic. We will use the exec command on kafka to execute a single kafka command. In this case we run the kafka-topics command with the --create option to create a topic. The --topic option is used to name the topic "events". I chose to name this topic events because we are collecting game event data for this project. We use the --partitions option to specify that we only need one partition. The --if-not-exists option is to prevent two topics with the same name being created. The --zookeeper option is specifying which port we would like to connect our topic to. To ensure this command is successful, the console will output "Created topic events".

```console
docker-compose exec kafka kafka-topics --create --topic events --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181
```

Moving on we will set up the game_api.py file which will define what type of events we can perform. To start, we will copy the game_api.py file from week 12 and then make some modifications. This is done using the command below.

```console
cp ~/w205/course-content/12-Querying-Data-II/game_api.py .
```

The game_api.py file from week 12 does not have functions for buy a sword or join guild so we will need to create them. Some neat functions of this project are that we give the user the option to buy a sword or buy a sharp sword. We also have 3 guilds the user can join to represent each team member on this project. There is the Morgan guild, the Lam guild, and the Kavlakoglu guild. There are separate events for each of these guilds which will join that specific guild while the generic join_guild event will randomly select a guild to join.

The functions added to the game_api.py file are shown below. These functions are very similar to the function that already exists in the game_api.py file named purchase_a_sword(). The first line of each function specifies the path that will need to be specified to trigger each event. In this case, we are specifying /buy_a_sword or /purchase_a_sharp_sword to trigger the a buy sword event. To join a guild, we specify /join_guild to join a random guild or the specific guild paths to choose which guild to join. Each of these functions defines a dictionary of metadata. All events have an event type. For buy_sword events, we specify the sword_type which is either normal or sharp. For join_guild events, we specify the guild_name, and the assignment_type which says if the guild was joined randomly or if we chose which guild we were joining. The log_to_kafka step sends the event and event metadata to the kafka topic "events". Finally the return statement returns a nice message to the user that tells them what happened. 

```python
@app.route("/buy_a_sword")
def buy_a_sword():
    buy_sword_event = {'event_type': 'buy_sword',
                       'sword_type': 'normal'}
    log_to_kafka('events', buy_sword_event)
    return "Sword Bought!\n"

@app.route("/join_guild")
def join_guild():
    guilds = ['Morgan','Lam','Kavlakoglu']
    guild = random.choice(guilds)
    join_guild_event = {'event_type': 'join_guild',
                        'guild_name': guild,
                        'assignment_type': 'random'}
    log_to_kafka('events', join_guild_event)
    return "Joined " + guild + " Guild!\n"

@app.route("/purchase_a_sharp_sword")
def purchase_a_sharp_sword():
    purchase_sword_event = {'event_type': 'buy_sword',
                            'sword_type': 'sharp'}
    log_to_kafka('events', purchase_sword_event)
    return "Sharp Sword Purchased!\n"

@app.route("/join_guild_morgan")
def join_guild_morgan():
    join_guild_event = {'event_type': 'join_guild',
                        'guild_name': 'Morgan',
                        'assignment_type': 'manual'}
    log_to_kafka('events', join_guild_event)
    return "Joined Morgan Guild!\n"

@app.route("/join_guild_lam")
def join_guild_lam():
    join_guild_event = {'event_type': 'join_guild',
                        'guild_name': 'Lam',
                        'assignment_type': 'manual'}
    log_to_kafka('events', join_guild_event)
    return "Joined Lam Guild!\n"

@app.route("/join_guild_kavlakoglu")
def join_guild_kavlakoglu():
    join_guild_event = {'event_type': 'join_guild',
                        'guild_name': 'Kavlakoglu',
                        'assignment_type': 'manual'}
    log_to_kafka('events', join_guild_event)
    return "Joined Kavlakoglu Guild!\n"
```

Now that we have updated our game_api.py file, we can run our flask app through the game_api.py file. The code to run our flask app already exists in the game_api file so we just need to run the file and specify the host which in this case is 0.0.0.0. 

```console
docker-compose exec mids env FLASK_APP=/w205/project-3-superpeter55/game_api.py flask run --host 0.0.0.0
```

Note that since the flask app is now running in the window. We will need to open a new terminal to perform commands. We do this and then switch into the proper directory using the commands below.

```console
cd w205
cd project-3-superpeter55/
```

The next task we would like to perform is spinning up a jupyter notebook with pyspark. Before we actually spin up the notebook, we must create a symbolic link so that we can access our mounted w205 directory. This is done using the commands below. The first line uses docker-compose exec to open up a spark bash shell. The second line creates link using the ln command and specifies the -s option to create a symbolic link. The third line exits and returns us to the shell.

```console
docker-compose exec spark bash
ln -s /w205 w205
exit 
```

Now we are ready to spin up our notebook using pyspark. This is done in the command below. Once again we are using docker-compose exec to run a single spark command. The spark command sets up a notebook in port 8888 and ip address 0.0.0.0. After this command we replace 0.0.0.0 with our local machine ip address and paste the link in our browser to access our notebooks.

```console
docker-compose exec spark env PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS='notebook --no-browser --port 8888 --ip 0.0.0.0 --allow-root' pyspark
```

Now that we are in our notebook, we want to check that the spark session and spark context are accessible in the notebook. The two code cells below show that spark is properly running in this instance and we are ready to proceed. We will move forward to testing our pipeline in batch mode and generating data with apache bench.

In [ ]:
spark

In [ ]:
sc

<SparkContext master=local[*] appName=PySparkShell>

## Batch Mode

### Generating Events

To test our pipeline, we will use Apache Bench to generate events and run them through the pipeline. To test our pipeline, we will generate 3 events of each type from 2 separate users which we will call, Player_1 and Player_2. Once again, we will use docker compose exec to excecute Apache Bench commands. The commands we will use to generate our data are listed below. Each event is very similar in that it call "ab" which means Apache Bench. The -n argument specifies how many times we would like to call this event which we specify as 3 for every single call to apache bench. The -H argument specifies the host or user which we have decided to specify either Player_1 or Player_2. Finally, the path at the end specifies which event from our game_api.py file will be called. I have decided to call every event in the api 3 times with each user to ensure that our test data covers all possible actions a user can take. Since we have 6 events in our api and 2 users generating test data, it makes sense that we have 12 calls to apache bench specified below. 

```console
docker-compose exec mids ab -n 3 -H "Host: Player_1" http://localhost:5000/
docker-compose exec mids ab -n 3 -H "Host: Player_2" http://localhost:5000/
docker-compose exec mids ab -n 3 -H "Host: Player_1" http://localhost:5000/purchase_a_sharp_sword
docker-compose exec mids ab -n 3 -H "Host: Player_2" http://localhost:5000/purchase_a_sharp_sword
docker-compose exec mids ab -n 3 -H "Host: Player_1" http://localhost:5000/buy_a_sword
docker-compose exec mids ab -n 3 -H "Host: Player_2" http://localhost:5000/buy_a_sword
docker-compose exec mids ab -n 3 -H "Host: Player_1" http://localhost:5000/join_guild
docker-compose exec mids ab -n 3 -H "Host: Player_2" http://localhost:5000/join_guild
docker-compose exec mids ab -n 3 -H "Host: Player_1" http://localhost:5000/join_guild_morgan
docker-compose exec mids ab -n 3 -H "Host: Player_2" http://localhost:5000/join_guild_morgan
docker-compose exec mids ab -n 3 -H "Host: Player_2" http://localhost:5000/join_guild_lam
docker-compose exec mids ab -n 3 -H "Host: Player_1" http://localhost:5000/join_guild_lam
docker-compose exec mids ab -n 3 -H "Host: Player_2" http://localhost:5000/join_guild_kavlakoglu
docker-compose exec mids ab -n 3 -H "Host: Player_1" http://localhost:5000/join_guild_kavlakoglu
```

In [ ]:
@udf('boolean')
def is_purchase(event_as_json):
    event = json.loads(event_as_json)
    if event['event_type'] == 'buy_sword':
        return True
    return False

@udf('boolean')
def is_join(event_as_json):
    event = json.loads(event_as_json)
    if event['event_type'] == 'join_guild':
        return True
    return False

@udf('boolean')
def is_default(event_as_json):
    event = json.loads(event_as_json)
    if event['event_type'] == 'default':
        return True
    return False

In [ ]:
raw_events = spark \
        .read \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "kafka:29092") \
        .option("subscribe", "events") \
        .option("startingOffsets", "earliest") \
        .option("endingOffsets", "latest") \
        .load()

In [ ]:
raw_events.show()

+----+--------------------+------+---------+------+--------------------+-------------+
| key|               value| topic|partition|offset|           timestamp|timestampType|
+----+--------------------+------+---------+------+--------------------+-------------+
|null|[7B 22 48 6F 73 7...|events|        0|     0|2021-12-08 03:32:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     1|2021-12-08 03:32:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     2|2021-12-08 03:32:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     3|2021-12-08 03:33:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     4|2021-12-08 03:33:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     5|2021-12-08 03:33:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     6|2021-12-08 03:33:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     7|2021-12-08 03:33:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0

In [ ]:
purchase_events = raw_events \
        .select(raw_events.value.cast('string').alias('raw'),
                raw_events.timestamp.cast('string')) \
        .filter(is_purchase('raw'))

In [ ]:
purchase_events.show()

+--------------------+--------------------+
|                 raw|           timestamp|
+--------------------+--------------------+
|{"Host": "Player_...|2021-12-08 03:33:...|
|{"Host": "Player_...|2021-12-08 03:33:...|
|{"Host": "Player_...|2021-12-08 03:33:...|
|{"Host": "Player_...|2021-12-08 03:33:...|
|{"Host": "Player_...|2021-12-08 03:33:...|
|{"Host": "Player_...|2021-12-08 03:33:...|
|{"Host": "Player_...|2021-12-08 03:33:...|
|{"Host": "Player_...|2021-12-08 03:33:...|
|{"Host": "Player_...|2021-12-08 03:33:...|
|{"Host": "Player_...|2021-12-08 03:33:...|
|{"Host": "Player_...|2021-12-08 03:33:...|
|{"Host": "Player_...|2021-12-08 03:33:...|
+--------------------+--------------------+



In [ ]:
extracted_purchase_events = purchase_events \
        .rdd \
        .map(lambda r: Row(timestamp=r.timestamp, **json.loads(r.raw))) \
        .toDF()

In [ ]:
extracted_purchase_events.printSchema()

root
 |-- Accept: string (nullable = true)
 |-- Host: string (nullable = true)
 |-- User-Agent: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- sword_type: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [ ]:
extracted_purchase_events.show()

+------+--------+---------------+----------+----------+--------------------+
|Accept|    Host|     User-Agent|event_type|sword_type|           timestamp|
+------+--------+---------------+----------+----------+--------------------+
|   */*|Player_1|ApacheBench/2.3| buy_sword|     sharp|2021-12-08 03:33:...|
|   */*|Player_1|ApacheBench/2.3| buy_sword|     sharp|2021-12-08 03:33:...|
|   */*|Player_1|ApacheBench/2.3| buy_sword|     sharp|2021-12-08 03:33:...|
|   */*|Player_2|ApacheBench/2.3| buy_sword|     sharp|2021-12-08 03:33:...|
|   */*|Player_2|ApacheBench/2.3| buy_sword|     sharp|2021-12-08 03:33:...|
|   */*|Player_2|ApacheBench/2.3| buy_sword|     sharp|2021-12-08 03:33:...|
|   */*|Player_1|ApacheBench/2.3| buy_sword|    normal|2021-12-08 03:33:...|
|   */*|Player_1|ApacheBench/2.3| buy_sword|    normal|2021-12-08 03:33:...|
|   */*|Player_1|ApacheBench/2.3| buy_sword|    normal|2021-12-08 03:33:...|
|   */*|Player_2|ApacheBench/2.3| buy_sword|    normal|2021-12-08 03:33:...|

In [ ]:
extracted_purchase_events \
        .write \
        .mode('overwrite') \
        .parquet('/tmp/purchases')

In [ ]:
join_events = raw_events \
        .select(raw_events.value.cast('string').alias('raw'),
                raw_events.timestamp.cast('string')) \
        .filter(is_join('raw'))
        
join_events.show()

+--------------------+--------------------+
|                 raw|           timestamp|
+--------------------+--------------------+
|{"event_type": "j...|2021-12-08 03:33:...|
|{"event_type": "j...|2021-12-08 03:33:...|
|{"event_type": "j...|2021-12-08 03:33:...|
|{"event_type": "j...|2021-12-08 03:33:...|
|{"event_type": "j...|2021-12-08 03:33:...|
|{"event_type": "j...|2021-12-08 03:33:...|
|{"event_type": "j...|2021-12-08 03:33:...|
|{"event_type": "j...|2021-12-08 03:33:...|
|{"event_type": "j...|2021-12-08 03:33:...|
|{"event_type": "j...|2021-12-08 03:33:...|
|{"event_type": "j...|2021-12-08 03:33:...|
|{"event_type": "j...|2021-12-08 03:33:...|
|{"event_type": "j...|2021-12-08 03:33:...|
|{"event_type": "j...|2021-12-08 03:33:...|
|{"event_type": "j...|2021-12-08 03:33:...|
|{"event_type": "j...|2021-12-08 03:34:...|
|{"event_type": "j...|2021-12-08 03:34:...|
|{"event_type": "j...|2021-12-08 03:34:...|
|{"event_type": "j...|2021-12-08 03:34:...|
|{"event_type": "j...|2021-12-08

In [ ]:
extracted_join_events = join_events \
        .rdd \
        .map(lambda r: Row(timestamp=r.timestamp, **json.loads(r.raw))) \
        .toDF()
        
extracted_join_events.printSchema()

root
 |-- Accept: string (nullable = true)
 |-- Host: string (nullable = true)
 |-- User-Agent: string (nullable = true)
 |-- assignment_type: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- guild_name: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [ ]:
extracted_join_events.show()

+------+--------+---------------+---------------+----------+----------+--------------------+
|Accept|    Host|     User-Agent|assignment_type|event_type|guild_name|           timestamp|
+------+--------+---------------+---------------+----------+----------+--------------------+
|   */*|Player_1|ApacheBench/2.3|         random|join_guild|       Lam|2021-12-08 03:33:...|
|   */*|Player_1|ApacheBench/2.3|         random|join_guild|    Morgan|2021-12-08 03:33:...|
|   */*|Player_1|ApacheBench/2.3|         random|join_guild|Kavlakoglu|2021-12-08 03:33:...|
|   */*|Player_2|ApacheBench/2.3|         random|join_guild|       Lam|2021-12-08 03:33:...|
|   */*|Player_2|ApacheBench/2.3|         random|join_guild|Kavlakoglu|2021-12-08 03:33:...|
|   */*|Player_2|ApacheBench/2.3|         random|join_guild|    Morgan|2021-12-08 03:33:...|
|   */*|Player_2|ApacheBench/2.3|         manual|join_guild|    Morgan|2021-12-08 03:33:...|
|   */*|Player_2|ApacheBench/2.3|         manual|join_guild|    Morgan

In [ ]:
extracted_join_events \
        .write \
        .mode('overwrite') \
        .parquet('/tmp/joins')

In [ ]:
default_events = raw_events \
        .select(raw_events.value.cast('string').alias('raw'),
                raw_events.timestamp.cast('string')) \
        .filter(is_default('raw'))
        
default_events.show()

+--------------------+--------------------+
|                 raw|           timestamp|
+--------------------+--------------------+
|{"Host": "Player_...|2021-12-08 03:32:...|
|{"Host": "Player_...|2021-12-08 03:32:...|
|{"Host": "Player_...|2021-12-08 03:32:...|
|{"Host": "Player_...|2021-12-08 03:33:...|
|{"Host": "Player_...|2021-12-08 03:33:...|
|{"Host": "Player_...|2021-12-08 03:33:...|
+--------------------+--------------------+



In [ ]:
extracted_default_events = default_events \
        .rdd \
        .map(lambda r: Row(timestamp=r.timestamp, **json.loads(r.raw))) \
        .toDF()
        
extracted_default_events.printSchema()

root
 |-- Accept: string (nullable = true)
 |-- Host: string (nullable = true)
 |-- User-Agent: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [ ]:
extracted_default_events.show()

+------+--------+---------------+----------+--------------------+
|Accept|    Host|     User-Agent|event_type|           timestamp|
+------+--------+---------------+----------+--------------------+
|   */*|Player_1|ApacheBench/2.3|   default|2021-12-08 03:32:...|
|   */*|Player_1|ApacheBench/2.3|   default|2021-12-08 03:32:...|
|   */*|Player_1|ApacheBench/2.3|   default|2021-12-08 03:32:...|
|   */*|Player_2|ApacheBench/2.3|   default|2021-12-08 03:33:...|
|   */*|Player_2|ApacheBench/2.3|   default|2021-12-08 03:33:...|
|   */*|Player_2|ApacheBench/2.3|   default|2021-12-08 03:33:...|
+------+--------+---------------+----------+--------------------+



In [ ]:
extracted_default_events \
        .write \
        .mode('overwrite') \
        .parquet('/tmp/defaults')

In [ ]:
purchases = spark.read.parquet('/tmp/purchases')
purchases.show()

+------+--------+---------------+----------+----------+--------------------+
|Accept|    Host|     User-Agent|event_type|sword_type|           timestamp|
+------+--------+---------------+----------+----------+--------------------+
|   */*|Player_1|ApacheBench/2.3| buy_sword|     sharp|2021-12-08 03:33:...|
|   */*|Player_1|ApacheBench/2.3| buy_sword|     sharp|2021-12-08 03:33:...|
|   */*|Player_1|ApacheBench/2.3| buy_sword|     sharp|2021-12-08 03:33:...|
|   */*|Player_2|ApacheBench/2.3| buy_sword|     sharp|2021-12-08 03:33:...|
|   */*|Player_2|ApacheBench/2.3| buy_sword|     sharp|2021-12-08 03:33:...|
|   */*|Player_2|ApacheBench/2.3| buy_sword|     sharp|2021-12-08 03:33:...|
|   */*|Player_1|ApacheBench/2.3| buy_sword|    normal|2021-12-08 03:33:...|
|   */*|Player_1|ApacheBench/2.3| buy_sword|    normal|2021-12-08 03:33:...|
|   */*|Player_1|ApacheBench/2.3| buy_sword|    normal|2021-12-08 03:33:...|
|   */*|Player_2|ApacheBench/2.3| buy_sword|    normal|2021-12-08 03:33:...|

In [ ]:
purchases.registerTempTable('purchases')

In [ ]:
purchases_player1 = spark.sql("select * from purchases where host='Player_1'")
purchases_player1.show()

+------+--------+---------------+----------+----------+--------------------+
|Accept|    Host|     User-Agent|event_type|sword_type|           timestamp|
+------+--------+---------------+----------+----------+--------------------+
|   */*|Player_1|ApacheBench/2.3| buy_sword|     sharp|2021-12-08 03:33:...|
|   */*|Player_1|ApacheBench/2.3| buy_sword|     sharp|2021-12-08 03:33:...|
|   */*|Player_1|ApacheBench/2.3| buy_sword|     sharp|2021-12-08 03:33:...|
|   */*|Player_1|ApacheBench/2.3| buy_sword|    normal|2021-12-08 03:33:...|
|   */*|Player_1|ApacheBench/2.3| buy_sword|    normal|2021-12-08 03:33:...|
|   */*|Player_1|ApacheBench/2.3| buy_sword|    normal|2021-12-08 03:33:...|
+------+--------+---------------+----------+----------+--------------------+



In [ ]:
joins = spark.read.parquet('/tmp/joins')
joins.show()

+------+--------+---------------+---------------+----------+----------+--------------------+
|Accept|    Host|     User-Agent|assignment_type|event_type|guild_name|           timestamp|
+------+--------+---------------+---------------+----------+----------+--------------------+
|   */*|Player_1|ApacheBench/2.3|         random|join_guild|       Lam|2021-12-08 03:33:...|
|   */*|Player_1|ApacheBench/2.3|         random|join_guild|    Morgan|2021-12-08 03:33:...|
|   */*|Player_1|ApacheBench/2.3|         random|join_guild|Kavlakoglu|2021-12-08 03:33:...|
|   */*|Player_2|ApacheBench/2.3|         random|join_guild|       Lam|2021-12-08 03:33:...|
|   */*|Player_2|ApacheBench/2.3|         random|join_guild|Kavlakoglu|2021-12-08 03:33:...|
|   */*|Player_2|ApacheBench/2.3|         random|join_guild|    Morgan|2021-12-08 03:33:...|
|   */*|Player_2|ApacheBench/2.3|         manual|join_guild|    Morgan|2021-12-08 03:33:...|
|   */*|Player_2|ApacheBench/2.3|         manual|join_guild|    Morgan

In [ ]:
joins.registerTempTable('joins')

In [ ]:
joins_player2 = spark.sql("select * from joins where host='Player_2'")
joins_player2.show()

+------+--------+---------------+---------------+----------+----------+--------------------+
|Accept|    Host|     User-Agent|assignment_type|event_type|guild_name|           timestamp|
+------+--------+---------------+---------------+----------+----------+--------------------+
|   */*|Player_2|ApacheBench/2.3|         random|join_guild|       Lam|2021-12-08 03:33:...|
|   */*|Player_2|ApacheBench/2.3|         random|join_guild|Kavlakoglu|2021-12-08 03:33:...|
|   */*|Player_2|ApacheBench/2.3|         random|join_guild|    Morgan|2021-12-08 03:33:...|
|   */*|Player_2|ApacheBench/2.3|         manual|join_guild|    Morgan|2021-12-08 03:33:...|
|   */*|Player_2|ApacheBench/2.3|         manual|join_guild|    Morgan|2021-12-08 03:33:...|
|   */*|Player_2|ApacheBench/2.3|         manual|join_guild|    Morgan|2021-12-08 03:33:...|
|   */*|Player_2|ApacheBench/2.3|         manual|join_guild|       Lam|2021-12-08 03:33:...|
|   */*|Player_2|ApacheBench/2.3|         manual|join_guild|       Lam

In [ ]:
defaults = spark.read.parquet('/tmp/defaults')
defaults.show()

+------+--------+---------------+----------+--------------------+
|Accept|    Host|     User-Agent|event_type|           timestamp|
+------+--------+---------------+----------+--------------------+
|   */*|Player_1|ApacheBench/2.3|   default|2021-12-08 03:32:...|
|   */*|Player_1|ApacheBench/2.3|   default|2021-12-08 03:32:...|
|   */*|Player_1|ApacheBench/2.3|   default|2021-12-08 03:32:...|
|   */*|Player_2|ApacheBench/2.3|   default|2021-12-08 03:33:...|
|   */*|Player_2|ApacheBench/2.3|   default|2021-12-08 03:33:...|
|   */*|Player_2|ApacheBench/2.3|   default|2021-12-08 03:33:...|
+------+--------+---------------+----------+--------------------+



## Streaming Mode

In [ ]:
def purchase_sword_event_schema():
    """
    root
    |-- Accept: string (nullable = true)
    |-- Host: string (nullable = true)
    |-- User-Agent: string (nullable = true)
    |-- event_type: string (nullable = true)
    |-- timestamp: string (nullable = true)
    """
    return StructType([
        StructField("Accept", StringType(), True),
        StructField("Host", StringType(), True),
        StructField("User-Agent", StringType(), True),
        StructField("event_type", StringType(), True),
        StructField("sword_type", StringType(), True)
    ])

In [ ]:
raw_events = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "kafka:29092") \
        .option("subscribe", "events") \
        .load()

In [ ]:
sword_purchases = raw_events \
        .filter(is_purchase(raw_events.value.cast('string'))) \
        .select(raw_events.value.cast('string').alias('raw_event'),
                raw_events.timestamp.cast('string'),
                from_json(raw_events.value.cast('string'),
                          purchase_sword_event_schema()).alias('json')) \
        .select('raw_event', 'timestamp', 'json.*')

In [ ]:
sink = sword_purchases \
        .writeStream \
        .format("parquet") \
        .option("checkpointLocation", "/tmp/checkpoints_for_sword_purchases") \
        .option("path", "/tmp/sword_purchases") \
        .trigger(processingTime="10 seconds") \
        .start()

Py4JJavaError: An error occurred while calling o313.start.
: java.lang.IllegalStateException: Cannot start query with id 41bdefa5-7dce-4628-b88f-26e229496251 as another query with same id is already active. Perhaps you are attempting to restart a query from checkpoint that is already active.
	at org.apache.spark.sql.streaming.StreamingQueryManager.startQuery(StreamingQueryManager.scala:300)
	at org.apache.spark.sql.streaming.DataStreamWriter.start(DataStreamWriter.scala:282)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:280)
	at py4j.commands.AbstractCommand.invokeMethod(Abstra

### Generating Events

Open third terminal, run while loop to continuously generate events. In this case, each player is going to perform every event the api has to offer once every 10 seconds. Once again, Apache Bench is used to generate the events. The only difference from the streaming version from the batch version is that we are specifying each event only happen once and we are also going to run each bench command once every 10 seconds so we are continuously streaming data.

```console
while true; do
    docker-compose exec mids ab -n 1 -H "Host: Player_1" http://localhost:5000/
    docker-compose exec mids ab -n 1 -H "Host: Player_2" http://localhost:5000/
    docker-compose exec mids ab -n 1 -H "Host: Player_1" http://localhost:5000/purchase_a_sharp_sword
    docker-compose exec mids ab -n 1 -H "Host: Player_2" http://localhost:5000/purchase_a_sharp_sword
    docker-compose exec mids ab -n 1 -H "Host: Player_1" http://localhost:5000/buy_a_sword
    docker-compose exec mids ab -n 1 -H "Host: Player_2" http://localhost:5000/buy_a_sword
    docker-compose exec mids ab -n 1 -H "Host: Player_1" http://localhost:5000/join_guild
    docker-compose exec mids ab -n 1 -H "Host: Player_2" http://localhost:5000/join_guild
    docker-compose exec mids ab -n 1 -H "Host: Player_1" http://localhost:5000/join_guild_morgan
    docker-compose exec mids ab -n 1 -H "Host: Player_2" http://localhost:5000/join_guild_morgan
    docker-compose exec mids ab -n 1 -H "Host: Player_2" http://localhost:5000/join_guild_lam
    docker-compose exec mids ab -n 1 -H "Host: Player_1" http://localhost:5000/join_guild_lam
    docker-compose exec mids ab -n 1 -H "Host: Player_2" http://localhost:5000/join_guild_kavlakoglu
    docker-compose exec mids ab -n 1 -H "Host: Player_1" http://localhost:5000/join_guild_kavlakoglu
  sleep 10
done
```

Now, we are ready to start Hive. this is done below.

```console
docker-compose exec cloudera hive
```

Write to HDFS in parquet format from hive

```console
create external table if not exists default.sword_purchases (Accept string, Host string, `User-Agent` string, event_type string, sword_size string, timestamp string) stored as parquet location '/tmp/sword_purchases'  tblproperties ("parquet.compress"="SNAPPY");
```

## Business Analytics Questions

Use sql in presto
how many events are there?
how many events from user1/user2?

In [ ]:
sink.stop()